In [16]:
from Threshold import binarize
from Morphology import erode, dilate
from Contour import keep_largest_contour
import os
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import matplotlib.pyplot as plt

# detekcja źrenicy

In [ ]:
image_raw = cv2.imread('eyes/21L1.bmp', 0)
image_bin = binarize(image_raw, threshold=0.22)

# czyszczenie
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
image = cv2.erode(image_bin, kernel, iterations=2)
image = cv2.dilate(image, kernel, iterations=3)
image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel, iterations=2)
image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel, iterations=2)
image = cv2.medianBlur(image, 5)
image = keep_largest_contour(image)

# projekcja
binary_image = (image > 0).astype(np.uint8)
horizontal_proj = np.sum(binary_image, axis=1)
vertical_proj = np.sum(binary_image, axis=0)

# środek
x = int(np.mean(np.where(vertical_proj == np.min(vertical_proj))[0]))
y = int(np.mean(np.where(horizontal_proj == np.min(horizontal_proj))[0]))

# promień
left_edge = np.min(np.where(vertical_proj > 0))
right_edge = np.max(np.where(vertical_proj > 0))
width_diameter = right_edge - left_edge
radius_horizontal = (right_edge - left_edge) // 2

top_edge = np.min(np.where(horizontal_proj > 0))
bottom_edge = np.max(np.where(horizontal_proj > 0))
height_diameter = bottom_edge - top_edge
radius_vertical = height_diameter // 2

radius = (radius_horizontal + radius_vertical) // 2

# zaznaczenie środka i promienia
image_center = cv2.cvtColor(image.copy(), cv2.COLOR_GRAY2BGR)
image_center = cv2.circle(image_center, (x, y), radius, (255, 0, 0), 2)
image_center = cv2.circle(image_center, (x, y), 0, (255, 0, 0), 4)

# wyświetlenie zdjęć i wykresów
plt.figure(figsize=(12, 8))
plt.subplot(2, 3, 1); plt.imshow(image_raw, cmap='gray'); plt.title('Original Image'); plt.axis('off')
plt.subplot(2, 3, 2); plt.imshow(image_bin, cmap='gray'); plt.title('Binarized Image'); plt.axis('off') 
plt.subplot(2, 3, 3); plt.imshow(image_center, cmap='gray'); plt.title('Pupil Detection with center'); plt.axis('off')
plt.subplot(2, 3, 4); plt.plot(horizontal_proj, color='red'); plt.title('Horizontal Projection')
plt.subplot(2, 3, 5); plt.plot(vertical_proj, color='red'); plt.title('Vertical Projection')
plt.show()

In [35]:
image_bin.shape

(240, 320)

In [ ]:
binary_image = (image > 0).astype(np.uint8)
horizontal_proj = np.sum(binary_image, axis=1)
vertical_proj = np.sum(binary_image, axis=0)


plt.figure(figsize=(12, 4))

# Projekcja pozioma (jako wykres liniowy)
plt.subplot(1, 2, 1)
plt.plot(horizontal_proj)
plt.title('Horizontal Projection')
plt.xlabel('Row index')
plt.ylabel('Sum of white pixels')

# Projekcja pionowa (jako wykres liniowy)
plt.subplot(1, 2, 2)
plt.plot(vertical_proj)
plt.title('Vertical Projection')
plt.xlabel('Column index')
plt.ylabel('Sum of white pixels')

plt.tight_layout()
plt.show()

# detekcja tęczówki

In [27]:
image_raw = cv2.imread('eyes/9L1.bmp', 0)
image_bin = binarize(image_raw, threshold=0.55)
small_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
big_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
image = cv2.dilate(image_bin, big_kernel, iterations=2)
image = cv2.erode(image, small_kernel, iterations=3)
image = keep_largest_contour(image)
image = cv2.erode(image, big_kernel, iterations=2)

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1); plt.imshow(image_raw, cmap='gray'); plt.title('Original Image'); plt.axis('off')
plt.subplot(1, 3, 2); plt.imshow(image_bin, cmap='gray'); plt.title('Binarized Image'); plt.axis('off') 
plt.subplot(1, 3, 3); plt.imshow(image, cmap='gray'); plt.title('Iris Detection'); plt.axis('off')
plt.show()